## HOMEWORK 1

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

/home/codespace/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/codespace/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


### Read the data for January. How many columns are there?

In [2]:
data = pd.read_parquet('/workspaces/mlops_stuff/dataset/yellow_tripdata_2023-01.parquet')
print(len(data.columns))

19


### What's the standard deviation of the trips duration in January?

In [3]:
data['duration'] = data['tpep_dropoff_datetime'] - data['tpep_pickup_datetime']
data['duration_minutes'] = data['duration'].dt.total_seconds() / 60
print(data['duration_minutes'].std())

42.59435124195458


### What fraction of the records left after you dropped the outliers?

In [4]:
data_clean = data[data['duration_minutes'].between(1.0, 60.0)]
difference = len(data['duration_minutes']) - len(data_clean['duration_minutes'])
print(f"We removed {(100*difference/len(data['duration_minutes'])):0.2f} % of the rows.")
print(f"So there is {100 - 1.88}% left.")

We removed 1.88 % of the rows.
So there is 98.12% left.


## Linear regression 

First step, split the data into training set and val set.

In [5]:
train_data, val_data = train_test_split(data_clean, test_size=0.2, random_state=42)

In [6]:
train_location_data = pd.DataFrame({
    'PULocationID': train_data['PULocationID'].apply(str),
    'DOLocationID': train_data['DOLocationID'].apply(str)
})
train_location_dicts = train_location_data.to_dict('records')

val_location_data = pd.DataFrame({
    'PULocationID': val_data['PULocationID'].apply(str),
    'DOLocationID': val_data['DOLocationID'].apply(str)
})
val_location_dicts = val_location_data.to_dict('records')


### Apply one hot encoding to PULocation and to DOLocation
One-hot encoding is a technique used to convert categorical data into a binary format where each category is represented by a separate column with a 1 indicating its presence and 0s for all other categories.

In [7]:
vectorizer = DictVectorizer(sparse=True)
X_train = vectorizer.fit_transform(train_location_dicts)
print(f"Shape of training matrix: {X_train.shape}")
print(f"Number of features: {len(vectorizer.get_feature_names_out())}")

X_val = vectorizer.transform(val_location_dicts)
print(f"Shape of val matrix: {X_val.shape}")


Shape of training matrix: (2407338, 513)
Number of features: 513
Shape of val matrix: (601835, 513)


In [8]:
y_train = train_data['duration_minutes'].values
y_val = val_data['duration_minutes'].values

In [9]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [10]:
train_pred = model.predict(X_train)
train_rmse = np.sqrt(mean_squared_error(y_train, train_pred))
print(f"RMSE on training data: {train_rmse:.4f} minutes")

RMSE on training data: 7.6481 minutes


In [11]:
val_pred = model.predict(X_val)
val_rmse = np.sqrt(mean_squared_error(y_val, val_pred))
print(f"RMSE on val data: {val_rmse:.4f} minutes")

RMSE on val data: 7.6567 minutes
